# Cluster analysis

In this notebook, we will look at how to debug blocked table similarities and the resulting clusters.

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# Assume that we have the "chartedIn" results
import takco

# tables = takco.HashBag._load('../../output/chartedIn/4-coltypes/*')
tables = takco.HashBag._load('../../output/chartedIn/5-cluster/*')

print(f"Got {sum(1 for _ in tables)} tables")

# Get biggest table
table = sorted(tables, key=lambda x: -x.get('numDataRows'))[0]
takco.preview( table )

Got 38 tables


?,0,1,2,3,4,5
,_pgTitle,Peak position Position,_Variable,Chart Chart (,Chart ) Chart,Chart
,These Days (Bon Jovi album),3,1995,,Argentine Albums Chart,
,These Days (Bon Jovi album),1,1995,,Australian Albums Chart,
,These Days (Bon Jovi album),1,1995,,Austrian Albums Chart,
,These Days (Bon Jovi album),3,1995,,Belgian Albums Chart (Flanders),
,These Days (Bon Jovi album),6,1995,,Belgian Albums Chart (Wallonia),


In [3]:
from takco.util import tableobj_to_dataframe
df = tableobj_to_dataframe(table)
for ci, col in enumerate(df.columns):
    print(f"{str(col):>35s}", dict( df.T.iloc[ci].value_counts()[:3] ))

                      ('_pgTitle',) {'Womanizer (song)': 64, '...Baby One More Time (song)': 50, 'Love. Angel. Music. Baby.': 48}
       ('Peak position\tPosition',) {'1': 212, '2': 74, '3': 69}
                     ('_Variable',) {'1995': 79, '2018': 60, '2012': 59}
                ('Chart\tChart (',) {'': 452, 'US Billboard 200': 29, 'UK Albums Chart': 21}
                ('Chart )\tChart',) {'': 735, 'US Billboard Hot 100': 19, 'Australian Albums Chart': 16}
                         ('Chart',) {'': 1150, 'Belgium': 4, 'Netherlands': 4}


In [4]:
takco.preview( table.get('partColAligns'), ntables=None )

?,0,1,2,3
∈,Q482994 Q2031291 Q208569,decimal,dateTime,string
,_pgTitle,Position,_Variable,Chart
,These Days (Bon Jovi album),3,1995,Argentine Albums Chart
,These Days (Bon Jovi album),1,1995,Australian Albums Chart
,These Days (Bon Jovi album),1,1995,Austrian Albums Chart
,These Days (Bon Jovi album),3,1995,Belgian Albums Chart (Flanders)
,These Days (Bon Jovi album),6,1995,Belgian Albums Chart (Wallonia)
?,0,1,2,3
∈,Q482994 Q208569 Q2031291,decimal,dateTime,Q43229 Q80793969 Q373899
,_pgTitle,Peak position,_Variable,Chart


In [5]:
from IPython.display import display
from takco.cluster.clustering import aggregate_similarities
import pandas as pd
sims = pd.read_csv('../../output/chartedIn/sims.csv', index_col=[0,1,2,3]).query('ti1 < ti2')
agg_func = '@mean(  @max(headjacc, @pow(headvec, 4)), @max(bodylsh, bodytype, @pow(bodyvec, 4) ) )'
sims['agg'] = aggregate_similarities(sims, agg_func)

partColAligns = table.get('partColAligns')
ci_head = {}
for p in partColAligns:
    head = [' '.join(c.get('text','') for c in hcol) for hcol in zip(*p['tableHeaders'])]
    l,g = p['partcol_local'], p['partcol_global']
    ci_head.update({ci:head[ l[pci] ] for pci,ci in g.items()})
ci_head = pd.Series(ci_head, name='header')

partsims = sims.join(ci_head, on='ci1', how='inner').join(ci_head, how='inner', on='ci2', rsuffix='2')

display(partsims.describe())

# partsims = partsims[partsims.header == '_pgTitle']
# partsims = partsims[partsims.header == 'Region']
# partsims = partsims[partsims['agg'] < .51]
partsims.sort_values('agg', ascending=False).head(60)

/export/scratch1/home/kruit/nary/takco/cluster/clustering.py:85: RuntimeWarning: All-NaN axis encountered
  "max": lambda *args: np.nanmax(args, axis=0),
/export/scratch1/home/kruit/nary/takco/cluster/clustering.py:87: RuntimeWarning: Mean of empty slice
  "mean": lambda *args: np.nanmean(args, axis=0),


,headjacc,headvec,bodylsh,bodyvec,bodytype,agg
count,44.000000,36.000000,52.000000,44.000000,120.000000,131.000000
mean,0.295455,0.703491,0.014573,0.700243,0.150281,0.231791
std,0.346380,0.268745,0.050234,0.199142,0.349125,0.354191
min,0.000000,0.385440,0.000000,0.209280,0.000000,0.000000
25%,0.000000,0.495630,0.000000,0.560835,0.000000,0.000000
50%,0.000000,0.658545,0.000000,0.732235,0.000000,0.030172
75%,0.500000,1.000000,0.000000,0.882038,0.000000,0.304621
max,1.000000,1.000000,0.300781,0.996080,1.000000,1.000000


headjacc  headvec   bodylsh  bodyvec  bodytype       agg  \
ti1 ti2 ci1 ci2                                                             
38  40  186 197       NaN      NaN       NaN      NaN  1.000000  1.000000   
5   40  26  197       NaN      NaN       NaN      NaN  1.000000  1.000000   
3   40  18  197       NaN      NaN       NaN      NaN  1.000000  1.000000   
5   40  25  195       1.0  1.00000       NaN      NaN  1.000000  1.000000   
3   38  19  189       0.5  1.00000  0.011719  0.88116  1.000000  1.000000   
    5   18  26        NaN      NaN       NaN      NaN  1.000000  1.000000   
5   38  26  186       NaN      NaN       NaN      NaN  1.000000  1.000000   
3   38  18  186       NaN      NaN       NaN      NaN  1.000000  1.000000   
        17  185       1.0  1.00000       NaN      NaN  1.000000  1.000000   
5   38  27  188       0.5  1.00000  0.000000  0.91380  0.999889  0.999944   
    40  27  198       1.0  1.00000  0.128906  0.96427  0.991262  0.995631   
38  40  188 198       0.5  1.00000  0.000000  0.94732  0.989187  0.994593   
3   5   19  27        1.0  1.00000  0.152344  0.99608  0.000000  0.992206   
        16  24        NaN      NaN  0.300781  0.98536  0.980770  0.980770   
    40  19  198       1.0  1.00000  0.089844  0.97423  0.000000  0.950418   
38  40  189 198       0.5  1.00000  0.023438  0.93191  0.000000  0.877108   
3   38  19  188       0.5  1.00000  0.003906  0.92792  0.000000  0.870691   
38  40  187 198       0.5  1.00000  0.000000  0.89329  0.000000  0.818376   
5   38  27  189       0.5  1.00000  0.011719  0.88090  0.000000  0.801076   
38  40  185 195       0.5  0.82146       NaN      NaN  1.000000  0.750000   
3   40  17  195       0.5  0.82146       NaN      NaN  1.000000  0.750000   
    5   17  25        0.5  0.82146       NaN      NaN  1.000000  0.750000   
5   38  25  185       0.5  0.82146       NaN      NaN  1.000000  0.750000   
3   38  19  187       0.5  1.00000  0.000000  0.83071  0.000000  0.738105   
5   38  27  187       0.5  1.00000  0.003906  0.80161  0.000000  0.706454   
        24  184       NaN      NaN  0.000000  0.89235  0.070499  0.634075   
3   38  16  184       NaN      NaN  0.019531  0.88467  0.103626  0.612527   
5   38  24  189       NaN      NaN  0.000000  0.79582  0.000000  0.401106   
3   5   16  27        NaN      NaN  0.000000  0.78292  0.000000  0.375725   
    40  16  194       NaN      NaN  0.011719  0.65852  0.366372  0.366372   
    5   19  24        NaN      NaN  0.000000  0.76921  0.000000  0.350090   
5   40  24  198       NaN      NaN  0.000000  0.75661  0.000000  0.327709   
    38  27  184       NaN      NaN  0.000000  0.74739  0.000000  0.312025   
3   38  16  189       NaN      NaN  0.000000  0.73836  0.000000  0.297216   
        19  184       NaN      NaN  0.000000  0.73350  0.000000  0.289468   
38  40  184 198       NaN      NaN  0.000000  0.73097  0.000000  0.285495   
            194       NaN      NaN  0.000000  0.50750  0.282843  0.282843   
3   40  16  198       NaN      NaN  0.000000  0.72279  0.000000  0.272928   
38  40  187 196       0.5      NaN  0.000000      NaN       NaN  0.250000   
        188 196       0.5      NaN  0.000000      NaN       NaN  0.250000   
        189 196       0.5      NaN  0.000000      NaN       NaN  0.250000   
5   40  24  194       NaN      NaN  0.000000  0.57379  0.249253  0.249253   
    38  24  187       NaN      NaN  0.000000  0.65954  0.000000  0.189219   
            188       NaN      NaN  0.000000  0.64804  0.000000  0.176363   
3   38  16  188       NaN      NaN  0.000000  0.60934  0.000000  0.137860   
            187       NaN      NaN  0.000000  0.60591  0.000000  0.134782   
5   40  27  197       NaN      NaN  0.000000  0.60244  0.000000  0.131721   
3   40  19  197       NaN      NaN  0.000000  0.59728  0.000000  0.127266   
38  40  189 197       NaN      NaN  0.000000  0.56484  0.000000  0.101789   
        187 197       NaN      NaN  0.000000  0.54882  0.000000  0.090723   
5   40  27  1